In [4]:
import os
import sys
import json
import time
import numpy as np
from sklearn.cluster import KMeans
from gensim.models import Word2Vec


I:\anaconda1\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
n_cluster = 10
X = np.load('../data/PCA/sen_vector_PCA.npy')
kmeans = KMeans(n_clusters=n_cluster).fit(X)
pred_label = kmeans.predict(X)


In [5]:
#####
ref = json.load(open('../data/preprocessing/standard_ref/standard_ref_without_effect.json'))

sentences = json.load(open('../data/feature_extraction/entity_pair_sentences.json'))

entity_names = open('../data/clean_data/entity_total.csv',encoding='utf8').read().split('\n')[:-1]

#####
path = '../data/model/noun_verb/300/model'
model = Word2Vec.load(path)
voca = list(model.wv.vocab)
model = ''


In [7]:
entity_dict = {}
for row in entity_names :
    entity_dict[row.split('\t')[1]] = row.split('\t')[0]

In [8]:
ok = 0
total = 0
ref_pair = []
for herb in ref :
    for drug in ref[herb] :
        if entity_dict[herb] in voca and entity_dict[drug] in voca :
            ref_pair.append([entity_dict[drug],entity_dict[herb]])


In [10]:
entity_pair = []
for each in sentences :
    entity_pair.append(sorted([each[1][each[0][0]],each[1][each[0][1]]]))

In [38]:
count = 0
result = {}
for each in ref_pair :
    if each in entity_pair :
        group = pred_label[entity_pair.index(each)]
        #if group == 9 :
            #print(each)
        try :
            result[group]+=1
        except :
            result[group] = 1
print(result)

{9: 16, 0: 4, 6: 1, 7: 1, 4: 1}


In [28]:
from scipy import spatial

##### cluster labeling - cosine similarity
centroids = kmeans.cluster_centers_
res = {}

for i in range(len(pred_label)) :
    cosine = 1 - spatial.distance.cosine(X[i],centroids[pred_label[i]] )
    if pred_label[i] not in res :
        res[pred_label[i]] = {cosine:i}
    else :
        res[pred_label[i]][cosine] = i


In [41]:
top =[ res[9][i] for i in sorted(res[9])[:5] ]
for i in top :
    print(sentences[i][1])
    print('\n')

['select', 'pressur', 'act', 'preserv', 'photosystem', 'marin', 'micro', 'drug_entity_3149', 'l', 'lineages.a', 'studi', 'conduct', 'herb_entity_7643', 'forest', 'produc', 'quantiti', 'mushroom', 'porcini', 'boletu', 'eduli', 'lato']


['result', 'obtain', 'impli', 'explan', 'origin', 'herb_entity_2909', 'comb', 'phase', 'term', 'drug_entity_10566', 'forc', 'discount', 'perform', 'investig', 'passiv', 'effect', 'properti', 'si', 'x', 'ge', 'nanowir']


['drug_entity_3149', 'l', 'effect', 'herb_entity_10078', 'extract', 'infus', 'microorgan', 'evalu', 'listeria', 'monocytogen', 'extract', 'sampl', 'analyz', 'control', 'show', 'effect', 'mg/ml', 'mg/l', 'concentr']


['lesion', 'model', 'pd', 'exploit', 'develop', 'gene', 'therapi', 'transplant', 'approaches.th', 'discoveri', 'receptor', 'drug_entity_458', 'compon', 'marijuana', 'open', 'horizon', 'exploit', 'herb_entity_6418', 'cannabinoid']


['find', 'transit', 'relat', 'select', 'sector', 'manifold', 'phase', 'obtain', 'map', 'manifo

In [77]:
def daviesbouldin(X, labels, centroids):

    import numpy as np
    from scipy.spatial.distance import pdist, euclidean

    nbre_of_clusters = len(centroids) #Get the number of clusters
    distances = [[] for e in range(nbre_of_clusters)] #Store intra-cluster distances by cluster
    distances_means = [] #Store the mean of these distances
    DB_indexes = [] #Store Davies_Boulin index of each pair of cluster
    second_cluster_idx = [] #Store index of the second cluster of each pair
    first_cluster_idx = 0 #Set index of first cluster of each pair to 0

    # Step 1: Compute euclidean distances between each point of a cluster to their centroid
    for cluster in range(nbre_of_clusters):
        for point in range(X[labels == cluster].shape[0]):
            distances[cluster].append(euclidean(X[labels == cluster][point], centroids[cluster]))

    # Step 2: Compute the mean of these distances
    for e in distances:
        distances_means.append(np.mean(e))

    # Step 3: Compute euclidean distances between each pair of centroid
    ctrds_distance = pdist(centroids) 

    # Tricky step 4: Compute Davies-Bouldin index of each pair of cluster   
    for i, e in enumerate(e for start in range(1, nbre_of_clusters) for e in range(start, nbre_of_clusters)):
        second_cluster_idx.append(e)
        if second_cluster_idx[i-1] == nbre_of_clusters - 1:
            first_cluster_idx += 1
        DB_indexes.append((distances_means[first_cluster_idx] + distances_means[e]) / ctrds_distance[i])

    # Step 5: Compute the mean of all DB_indexes   
    print("DAVIES-BOULDIN Index: %.5f" % np.mean(DB_indexes)) 
    return np.mean(DB_indexes)

In [78]:
davie = {}
calinski = {}
X = np.load('../data/PCA/sen_vector_PCA.npy')
for n_cluster in range(2,50) :
    kmeans = KMeans(n_clusters=n_cluster, max_iter=1000).fit(X)
    labels = kmeans.predict(X)
    centroids = kmeans.cluster_centers_
    davie[n_cluster] = daviesbouldin(X, labels, centroids)
    calinski[n_cluster] =  sklearn.metrics.calinski_harabaz_score(X,labels)
    

DAVIES-BOULDIN Index: 2.15218
DAVIES-BOULDIN Index: 1.57891
DAVIES-BOULDIN Index: 1.64996
DAVIES-BOULDIN Index: 1.67187
DAVIES-BOULDIN Index: 1.68358
DAVIES-BOULDIN Index: 1.23870
DAVIES-BOULDIN Index: 1.25308
DAVIES-BOULDIN Index: 1.28893
DAVIES-BOULDIN Index: 1.28271
DAVIES-BOULDIN Index: 1.30754
DAVIES-BOULDIN Index: 1.33104
DAVIES-BOULDIN Index: 1.32987
DAVIES-BOULDIN Index: 1.34020
DAVIES-BOULDIN Index: 1.32118
DAVIES-BOULDIN Index: 1.32412
DAVIES-BOULDIN Index: 1.33698
DAVIES-BOULDIN Index: 1.33346
DAVIES-BOULDIN Index: 1.34397
DAVIES-BOULDIN Index: 1.34838
DAVIES-BOULDIN Index: 1.34047
DAVIES-BOULDIN Index: 1.32625
DAVIES-BOULDIN Index: 1.32545
DAVIES-BOULDIN Index: 1.32743
DAVIES-BOULDIN Index: 1.31306
DAVIES-BOULDIN Index: 1.32498
DAVIES-BOULDIN Index: 1.29528
DAVIES-BOULDIN Index: 1.32168
DAVIES-BOULDIN Index: 1.29037
DAVIES-BOULDIN Index: 1.31982
DAVIES-BOULDIN Index: 1.28982
DAVIES-BOULDIN Index: 1.29385
DAVIES-BOULDIN Index: 1.27049
DAVIES-BOULDIN Index: 1.28276
DAVIES-BOU